In [1]:
from unsloth import FastLanguageModel
import unsloth
import torch
import warnings

# suppresses some noisey warnings which are just annoying
warnings.filterwarnings('ignore')
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = None, load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Llama patching. Transformers = 4.46.0.dev0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
# PEFT MODEL
model = FastLanguageModel.get_peft_model(
    model,
     # Suggested choice of 8, 16, 32, 64, or 128
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",]
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def format_prompts(p):
    # these are provided as lists
    instructions = p["instruction"]
    inputs       = p["input"]
    outputs      = p["output"]
    texts = []
    for ins, inp, outp in zip(instructions, inputs, outputs):
        text = prompt.format(ins, inp, outp) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts }


In [4]:
import re
import json
import datasets
from bs4 import BeautifulSoup

# re-format dataset

#ins = '''
#Think very carefully about the question posed, and choose from one of the following:
#1. Write a blog about the topic
#2. Explain the technical concepts in detail
#3. Provide examples where possible.
#4. Provide as much about the author of any source blog material as possible.
#'''
ins = ''

# process blog data
with open('bhis-blogs.json', 'rt') as fh:
    data = json.load(fh)

# re-format dataset
newds = []
for i, item in enumerate(data):
    s = BeautifulSoup(item['content'], "html.parser")
    content = s.get_text()
    content = re.sub(r'\s+', ' ', content)
    if len(content) < 512:
        continue
    if len(content) > max_seq_length:
        for line in content.split('\n'):
            if line:
                newds.append({'input': '', 'instruction': ins, 'output': f"{item['title']} {' '.join(item['taxonomies'])} {line}"})
    else:
        newds.append({'input': '', 'instruction': ins, 'output': f"{item['title']} {' '.join(item['taxonomies'])} {content}"})

ds = datasets.Dataset.from_list(newds)
ds = ds.map(format_prompts, batched=True)

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

newmodel = model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        # Floating Point 16 (2 bytes memory use)
        fp16 = not is_bfloat16_supported(),
        # Brain Float 16 (2 bytes memory use but more efficient)
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 455 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.372400
2,2.282500
3,2.414000
4,2.407800
5,2.078900
6,2.460600
7,2.267800
8,2.272000
9,2.370400
10,2.345100


In [7]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory/max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
9.336 GB of memory reserved.
534.3228 seconds used for training.
8.91 minutes used for training.
Peak reserved memory = 9.336 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 38.919 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [8]:
model.save_pretrained('bhisblogs-llama3')


In [10]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format('''
Describe the best possible password cracking techniques.
''', "", "")
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 8192)

<|begin_of_text|>
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:

Describe the best possible password cracking techniques.


### Input:


### Response:
The best possible password cracking techniques are as follows: 1. Dictionary attacks: This is the most common and simplest password cracking technique. It involves using a list of commonly used passwords, also known as a dictionary, to guess the password for a given account. The dictionary can be generated from publicly available sources such as leaked password databases or by analyzing common patterns in passwords. 2. Brute force attacks: This technique involves trying every possible combination of characters in a password until the correct one is found. It is a time-consuming process, but it can be effective if the password is not complex enough or if the attacker has access to a large amount of computing po